# 4.2 Autograd

- Back Propagation 방식: Autograd로 설정되어 있음

In [1]:
import torch

## torch.cuda.is_available()
## 현재 실행되고 있는 환경에서
## torch module를 이용할 때 GPU를 이용할 수 있는지 파악
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')

print(DEVICE)

cuda


## 4.2.1 Input, Ouput, Weight 설정

In [2]:
BATCH_SIZE = 64    # model에서 parameter를 업데이트할 때 계산되는 데이터의 개수
INPUT_SIZE = 1000  # Input의 크기이자 입력층의 노드 개수
HIDDEN_SIZE = 100  # Input layer에서 hidden layer로 전달되었을 때 hidden layer의 node 수
OUTPUT_SIZE = 10   # 최종 출력 벡터의 크기, 보통 최종으로 비교하고자 하는 label의 크기와 동일하게 설정됨

#### input (x) → hidden layer1 (w1 \* x) → hidden layer2 (w2 \* (w1 \* x)) → output (y)

In [3]:
## torch.randn()
## Input x
## 평균:0, 표준편차가 1인 정규분포에서 샘플링한 값으로 데이터를 만듦
## BATCH_SIZE, INPUT_SIZE: 크기가 INPUT_SIZE인 벡터를 BATCH_SIZE만큼 생성
## device: 미리 설정한 DEVICE 이용
## float: 데이터 형태
## requires_grad = False: Input으로 이용되기 때문에 Gradient를 계산할 필요가 없음
x = torch.randn(BATCH_SIZE,
                INPUT_SIZE,
                device = DEVICE,
                dtype = torch.float,
                requires_grad = False
               )

In [4]:
## Output y
y = torch.randn(BATCH_SIZE,
                OUTPUT_SIZE,
                device = DEVICE,
                dtype = torch.float,
                requires_grad = False
               )

In [5]:
## Weight w1
w1 = torch.randn(INPUT_SIZE,
                 HIDDEN_SIZE,
                 device = DEVICE,
                 dtype = torch.float,
                 requires_grad = True
                )

In [6]:
## Weight w2
w2 = torch.randn(HIDDEN_SIZE,
                 OUTPUT_SIZE,
                 device = DEVICE,
                 dtype = torch.float,
                 requires_grad = True
                )

## 4.2.2 Forward pass & Backward pass

In [7]:
learning_rate = 1e-6
for t in range(1, 501):
    # .mm: matrix multiplication
    # .clamp(min=0): activation func, ReLU
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    # loss: 오차의 제곱합
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 0:
        print(f'Iteration: {t}\t Loss: {loss.item()}')
    
    # backpropagation
    loss.backward()
    
    # 각 parameter 값에 대해 gradient를 계산한 결과를 이용해
    # parameter 값을 업데이트할 때는 해당 시점의 gradient 값을 고정한 후 업데이트 진행
    # 코드가 실행되는 시점에서 gradient 값을 고정한다는 의미
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # parameter 값을 업데이트 했다면
        # 각 parameter 값의 gradient를 0으로 초기화해 다음 반복문이 진행될 수 있도록 함
        w1.grad.zero_()
        w2.grad.zero_()

Iteration: 100	 Loss: 401.73162841796875
Iteration: 200	 Loss: 1.3751574754714966
Iteration: 300	 Loss: 0.00900244526565075
Iteration: 400	 Loss: 0.0002254527062177658
Iteration: 500	 Loss: 3.734287747647613e-05
